<a href="https://colab.research.google.com/github/rahulbharathi/NLP_sentiment_analysis/blob/main/NLP_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

In [ ]:
uploaded = files.upload()

Saving Zomato restaurant reviews.csv to Zomato restaurant reviews.csv


In [ ]:
#from google.colab import drive

In [ ]:
#drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
data = pd.read_csv("Zomato restaurant reviews.csv", encoding='latin')
data.head()

,Restaurant,Reviewer,Review,Rating,Metadata,Time,Pictures
0,Beyond Flavours,Rusha Chakraborty,"The ambience was good, food was quite good . h...",5,"1 Review , 2 Followers",5/25/2019 15:54,0
1,Beyond Flavours,Anusha Tirumalaneedi,Ambience is too good for a pleasant evening. S...,5,"3 Reviews , 2 Followers",5/25/2019 14:20,0
2,Beyond Flavours,Ashok Shekhawat,A must try.. great food great ambience. Thnx f...,5,"2 Reviews , 3 Followers",5/24/2019 22:54,0
3,Beyond Flavours,Swapnil Sarkar,Soumen das and Arun was a great guy. Only beca...,5,"1 Review , 1 Follower",5/24/2019 22:11,0
4,Beyond Flavours,Dileep,Food is good.we ordered Kodi drumsticks and ba...,5,"3 Reviews , 2 Followers",5/24/2019 21:37,0


In [ ]:
data.drop(['Restaurant', 'Reviewer', 'Metadata', 'Time', 'Pictures'], axis =1, inplace = True)

In [ ]:
data.dropna(inplace = True)

In [ ]:
import nltk

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
#from nltk.tokenize import RegexpTokenizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import string

In [ ]:
def remove_punc_2(string_input):
    try:
        no_punc = "".join([char for char in string_input if char not in string.punctuation])
    except:
        print(string_input)
    return no_punc

In [ ]:
data['clean_text'] = data.Review.apply(remove_punc_2)

In [ ]:
data.head()

,Review,Rating,clean_text
0,"The ambience was good, food was quite good . h...",5,The ambience was good food was quite good had...
1,Ambience is too good for a pleasant evening. S...,5,Ambience is too good for a pleasant evening Se...
2,A must try.. great food great ambience. Thnx f...,5,A must try great food great ambience Thnx for ...
3,Soumen das and Arun was a great guy. Only beca...,5,Soumen das and Arun was a great guy Only becau...
4,Food is good.we ordered Kodi drumsticks and ba...,5,Food is goodwe ordered Kodi drumsticks and bas...


In [ ]:
data['clean_text_lower'] = data['clean_text'].apply(lambda x: " ".join([str(word.lower()) for word in x.split()]))

In [ ]:
data.head()

,Review,Rating,clean_text,clean_text_lower
0,"The ambience was good, food was quite good . h...",5,The ambience was good food was quite good had...,the ambience was good food was quite good had ...
1,Ambience is too good for a pleasant evening. S...,5,Ambience is too good for a pleasant evening Se...,ambience is too good for a pleasant evening se...
2,A must try.. great food great ambience. Thnx f...,5,A must try great food great ambience Thnx for ...,a must try great food great ambience thnx for ...
3,Soumen das and Arun was a great guy. Only beca...,5,Soumen das and Arun was a great guy Only becau...,soumen das and arun was a great guy only becau...
4,Food is good.we ordered Kodi drumsticks and ba...,5,Food is goodwe ordered Kodi drumsticks and bas...,food is goodwe ordered kodi drumsticks and bas...


In [ ]:
stopword_master = stopwords.words('english')
data['clean_text_lower_no_stop'] = data['clean_text_lower'].apply(lambda x:  " ".join([word for word in x.split() if not word in stopword_master]))

In [ ]:
stemmer = PorterStemmer()
def stemmer_func(sent):
    final = " ".join([stemmer.stem(word) for word in sent.split()])
    return final

In [ ]:
data['clean_stem_word'] = data.clean_text_lower_no_stop.apply(stemmer_func)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(max_features=10000, ngram_range  = (1,2),max_df = .95, min_df = 4)

vect.fit(data['clean_stem_word'])

sparse = vect.transform(data['clean_stem_word'])

In [ ]:
sparse

<9955x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 302907 stored elements in Compressed Sparse Row format>

In [ ]:
spare_pd = sparse.toarray()

In [ ]:
finaldata = pd.DataFrame(spare_pd)

In [ ]:
finaldata.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,9960,9961,9962,9963,9964,9965,9966,9967,9968,9969,9970,9971,9972,9973,9974,9975,9976,9977,9978,9979,9980,9981,9982,9983,9984,9985,9986,9987,9988,9989,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(finaldata, data['Rating'],test_size = 0.2, random_state = 12121)

In [ ]:
rf_model = RandomForestClassifier()
param_grid = {"n_estimators" : [75, 100], "max_depth" : [30,40], "criterion" : ["gini", "entropy"]}

GSCV = GridSearchCV( estimator= rf_model, param_grid = param_grid,  n_jobs = -1, verbose =3)

In [ ]:
GSCV.fit(train_X,train_y)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  4.7min
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  6.8min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              ra

In [ ]:
rf_pred = GSCV.predict(test_X)

In [ ]:
GSCV.best_estimator_

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=40, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
GSCV.best_score_

0.5698133637427564

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(rf_pred, test_y))

              precision    recall  f1-score   support

           1       0.74      0.75      0.75       363
         1.5       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
         2.5       0.00      0.00      0.00         0
           3       0.03      0.60      0.05        10
         3.5       0.00      0.00      0.00         0
           4       0.32      0.41      0.36       355
         4.5       0.00      0.00      0.00         0
           5       0.91      0.55      0.69      1263
        Like       0.00      0.00      0.00         0

    accuracy                           0.56      1991
   macro avg       0.20      0.23      0.18      1991
weighted avg       0.77      0.56      0.64      1991



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
kn = KNeighborsClassifier(n_neighbors= 9)

In [ ]:
kn.fit(train_X, train_y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=9, p=2,
                     weights='uniform')

In [ ]:
kn_pred = kn.predict(test_X)

In [ ]:
print(classification_report(kn_pred, test_y))

              precision    recall  f1-score   support

           1       0.15      0.24      0.19       241
         1.5       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         7
         2.5       0.00      0.00      0.00         0
           3       0.01      0.33      0.02         6
         3.5       0.00      0.00      0.00         0
           4       0.00      0.67      0.01         3
         4.5       0.00      0.00      0.00         0
           5       0.90      0.40      0.55      1734
        Like       0.00      0.00      0.00         0

    accuracy                           0.38      1991
   macro avg       0.11      0.16      0.08      1991
weighted avg       0.80      0.38      0.50      1991



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(kn_pred, test_y)

array([[ 57,   0,  23,   0,  28,   0,  54,   2,  76,   1],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   3,   0,   3,   0,   1,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   1,   0,   2,   0,   3,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   2,   0,   1,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [311,   4, 115,   1, 200,  10, 394,  10, 689,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0]])

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
ada = AdaBoostClassifier(base_estimator = rf_model, n_estimators =25)

In [ ]:
ada.fit(train_X, train_y)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=RandomForestClassifier(bootstrap=True,
                                                         ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='gini',
                                                         max_depth=None,
                                                         max_features='auto',
                                                         max_leaf_nodes=None,
                                                         max_samples=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                       

In [ ]:
ada_pred = ada.predict(test_X)

In [ ]:
print(classification_report(ada_pred, test_y))

NameError: ignored

In [ ]:
from sklearn.svm import SVC 

In [ ]:
svc_model = SVC()

In [ ]:
param_grid_svc = {"kernel" : ["sigmoid", "linear", "rbf"], "C" : [0.01, 0.1, 1]}

In [ ]:
GSV_svc = GridSearchCV(estimator = svc_model, param_grid = param_grid_svc, cv=5, n_jobs =-1, verbose =3)

NameError: ignored

In [ ]:
GSV_svc.fit(train_X, train_y)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


In [ ]:
gsv_pred = GSV_svc.predict(test_x)

NameError: ignored